# Non-negative Matrix Factorization

This notebook explores NMF on the 20-newsgroups dataset.

We optimize the following problem using projected gradient descent:
$$min_{W,H}\,||X-WH^T||^2_F,\, subject\, to\, W >= 0, H >= 0 $$

In [180]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from pandas import DataFrame
from IPython.display import display

In [181]:
newsgroups_train = fetch_20newsgroups(subset='train')
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X = vectorizer.fit_transform(newsgroups_train.data)
print X.shape

(11314, 10000)


In [182]:
k = 20
max_iters = 100

In [183]:
avg = np.sqrt(X.mean() / k)
H = avg * np.random.randn(X.shape[1],k)
W = avg * np.random.randn(X.shape[0], k)
np.abs(H, H)
np.abs(W, W)

array([[ 0.00243964,  0.00614835,  0.00766246, ...,  0.002906  ,
         0.00299917,  0.00891053],
       [ 0.00429421,  0.00255024,  0.00053396, ...,  0.00017203,
         0.0043647 ,  0.00401884],
       [ 0.01135328,  0.00157241,  0.00345792, ...,  0.00089553,
         0.00140209,  0.00086611],
       ..., 
       [ 0.00334875,  0.00539204,  0.00959739, ...,  0.00143813,
         0.00297352,  0.00703612],
       [ 0.0006364 ,  0.00125523,  0.0028788 , ...,  0.00848268,
         0.00123131,  0.00189993],
       [ 0.01145108,  0.00181263,  0.00580595, ...,  0.0077152 ,
         0.01135805,  0.00437633]])

# Train NMF Model

In [184]:
def d_dH(X, W, H):
    return -X.T.dot(W) + H.dot(W.T.dot(W))

def d_dW(X, W, H):
    return -X.dot(H)+W.dot(H.T.dot(H))

def objective(X, W, H):
    inner = X-W.dot(H.T)
    return .5*np.linalg.norm(inner)**2

In [185]:
W = W0.copy()
H = H0.copy()
learning_rate = 0.01
gamma = 0.1
print W.shape, H.shape

(11314, 20) (10000, 20)


In [186]:
print 'loss: %.4f' % (objective(X,W,H))
for _ in range(max_iters):
    # Fit W
    gradW = d_dW(X,W,H)
    
    learning_rate = 1.
    for j in range(20):
        m = objective(X,W,H) +\
            (learning_rate)*(np.linalg.norm(gradW)**2)
        if objective(X,W-learning_rate*gradW,H) > m:
            learning_rate = gamma*learning_rate
        else:
            break
    W = np.maximum(0, W.copy()-learning_rate*gradW.copy())
    # Fit H
    gradH = d_dH(X,W,H) 
    
    learning_rate = 1.
    for j in range(20):
        m = objective(X,W,H) +\
            (learning_rate)*(np.linalg.norm(gradH)**2)
        if objective(X,W,H-learning_rate*gradH) > m:
            learning_rate = gamma*learning_rate
        else:
            break
    H = np.maximum(0, H.copy()-learning_rate*gradH.copy())
    #learning_rate = learning_rate/2.
    #print H[0:1], gradH[:1]
    print 'loss: %.4f, learning_rate: %.8f' % (np.sqrt(objective(X,W,H)), learning_rate)
    

loss: 2085023997.9534
loss: 290.7963, learning_rate: 0.00010000
loss: 85.9110, learning_rate: 0.00100000
loss: 75.8782, learning_rate: 0.01000000
loss: 74.7446, learning_rate: 0.01000000
loss: 74.6008, learning_rate: 0.01000000
loss: 74.5095, learning_rate: 0.01000000
loss: 74.4762, learning_rate: 0.01000000
loss: 74.4518, learning_rate: 0.01000000
loss: 74.4336, learning_rate: 0.01000000
loss: 74.4181, learning_rate: 0.01000000
loss: 74.4038, learning_rate: 0.01000000
loss: 74.3411, learning_rate: 0.01000000
loss: 74.3156, learning_rate: 0.01000000
loss: 74.4205, learning_rate: 0.10000000
loss: 74.0958, learning_rate: 0.01000000
loss: 74.0447, learning_rate: 0.01000000
loss: 73.8900, learning_rate: 0.10000000
loss: 73.9451, learning_rate: 0.10000000
loss: 74.8334, learning_rate: 0.10000000
loss: 73.6904, learning_rate: 0.10000000
loss: 73.5132, learning_rate: 0.10000000
loss: 73.3788, learning_rate: 0.10000000
loss: 73.2761, learning_rate: 0.10000000
loss: 73.1880, learning_rate: 0.10

# Analysis

In [187]:
word_lookup = {}
for k,v in vectorizer.vocabulary_.iteritems():
    word_lookup[v] = k

In [188]:
topic_index = 0
topic_matrix = []
for topic in H.T:
    top_words = topic.argsort()[::-1][:5]
    cnt = 0
    tmp = []
    for x in top_words:
        #print topic_index, word_lookup[x]
        #print cnt, topic_index
        tmp.append(word_lookup[x])
    topic_matrix.append(tmp)
    topic_index += 1

df = DataFrame(np.array(topic_matrix).T)
display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,scsi,ca,nasa,msg,columbia,georgia,edu,uk,pitt,god,access,people,keith,cmu,israel,key,windows,com,sandvik,cleveland
1,drive,team,gov,food,cc,ai,university,ac,geb,jesus,digex,gun,caltech,andrew,israeli,clipper,window,netcom,apple,cwru
2,ide,game,space,dyer,gld,uga,cs,__,gordon,bible,pat,don,sgi,mellon,armenian,chip,dos,hp,kent,freenet
3,controller,hockey,jpl,sensitivity,cunixb,michael,state,dcs,banks,christian,net,think,livesey,carnegie,turkish,encryption,file,ibm,newton,ins
4,drives,players,henry,chinese,dare,covington,posting,university,cs,christians,express,guns,morality,pittsburgh,armenians,keys,files,article,com,reserve


In [189]:
print H.min(), H.max()

0.0 2.13111523585


In [190]:
W[:,9].argmax()

5559

In [191]:
print newsgroups_train.data[5559]

From: davem@bnr.ca (Dave Mielke)
Subject: Does God love you?
Organization: Bell Northern Research, Ottawa, Canada
Lines: 416

I have come across what I consider to be an excellent tract. It is a
bit lengthy for a posting, but I thought I'd share it with all of you
anyway. Feel free to pass it along to anyone whom you feel might
benefit from what it says. May God richly bless those who read it.
 
 
                   D O E S  G O D  L O V E  Y O U ?
 
 
Q. What  kind  of  question  is that?   Anyone who can read sees signs,
   tracts, books, and bumper stickers that say, "God Loves You."  Isn't
   that true?
 
A. It  is  true that God offers His love to the whole world, as we read
   in one of the most quoted verses in the Bible:
 
      For  God  so  loved  the world, that he gave his only begotten
      Son, that whosoever believeth in him should  not  perish,  but
      have everlasting life.                               John 3:16
 
   However, God's love is qualified.  The Bible sa